In [1]:
import pandas as pd
import numpy as np

# Indlæs filen
df_landbrug = pd.read_csv('Data/landbrugsdata.csv')

#filter for løbende priser og foregående års priser
df_landbrug_loebende=df_landbrug.loc[df_landbrug['PRISENHED']=='Løbende priser']
df_landbrug_loebende.drop(columns=['PRISENHED'],inplace=True)

df_landbrug_sum=df_landbrug_loebende.loc[df_landbrug_loebende['ANVENDELSE']=='Anvendelse, i alt-(Anvendelse)']
df_landbrug_loebende=df_landbrug_loebende.loc[df_landbrug_loebende['ANVENDELSE']!='Anvendelse, i alt-(Anvendelse)']

df_landbrug_sum

C:\Users\b431385\AppData\Local\Temp\ipykernel_26308\475108940.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_landbrug_loebende.drop(columns=['PRISENHED'],inplace=True)


,TILGANG1,TILGANG2,ANVENDELSE,TID,INDHOLD
8,Dansk produktion,100010 Slagterier-(Tilgang),"Anvendelse, i alt-(Anvendelse)",1967,6862.242
12,Import inkl. told,010000 Landbrug og gartneri-(Tilgang),"Anvendelse, i alt-(Anvendelse)",1987,7011.063
32,Import inkl. told,100030 Mejerier-(Tilgang),"Anvendelse, i alt-(Anvendelse)",1987,734.377
42,Dansk produktion,100010 Slagterier-(Tilgang),"Anvendelse, i alt-(Anvendelse)",1988,32924.229
62,Dansk produktion,100030 Mejerier-(Tilgang),"Anvendelse, i alt-(Anvendelse)",1967,3597.858
...,...,...,...,...,...
7783,Told,"100040x100050 Anden fødevareindustri (100040, ...","Anvendelse, i alt-(Anvendelse)",2018,206.807
7784,Told,"100040x100050 Anden fødevareindustri (100040, ...","Anvendelse, i alt-(Anvendelse)",2019,211.286
7785,Told,"100040x100050 Anden fødevareindustri (100040, ...","Anvendelse, i alt-(Anvendelse)",2020,230.819
7786,Told,"100040x100050 Anden fødevareindustri (100040, ...","Anvendelse, i alt-(Anvendelse)",2021,238.171


In [2]:
# Beregn summen af alle de enkelte anvendelser
beregnet_sum = df_landbrug_loebende.groupby(['TILGANG1', 'TILGANG2', 'TID'])['INDHOLD'].sum().reset_index()

# Omdøb kolonnen så vi kan kende forskel
beregnet_sum.rename(columns={'INDHOLD': 'BEREGNET_SUM'}, inplace=True)


In [3]:
# Vi merger de to tabeller
tjek_df = pd.merge(
    df_landbrug_sum, 
    beregnet_sum, 
    on=['TILGANG1', 'TILGANG2', 'TID']
)

# Beregn forskellen
tjek_df['DIFFERENCE'] = tjek_df['INDHOLD'] - tjek_df['BEREGNET_SUM']

# Find de rækker hvor forskellen er større end en bagatel (f.eks. 0.01 pga. afrunding)
fejl = tjek_df[tjek_df['DIFFERENCE'].abs() > 10]

if fejl.empty:
    print("Succes! Alle rækker summerer perfekt til 'Anvendelse, i alt'.")
else:
    print(f"OBS: Der er fundet {len(fejl)} rækker, der ikke stemmer.")
    display(fejl.head())

fejl

Succes! Alle rækker summerer perfekt til 'Anvendelse, i alt'.


,TILGANG1,TILGANG2,ANVENDELSE,TID,INDHOLD,BEREGNET_SUM,DIFFERENCE
